# 7. 代理

<div class="toc">
    <ul class="toc-item">
    <li><span><a href="#7.1-LangChain内置工具" data-toc-modified-id="7.1-LangChain内置工具-1">7.1 LangChain内置工具</a></span>
        <ul class="toc-item">
            <li><span><a href="#7.1.1-使用llm-math和wikipedia工具" data-toc-modified-id="7.1.1-使用llm-math和wikipedia工具-1.1">7.1.1 使用llm-math和wikipedia工具</a></span>
            <li><span><a href="#7.1.2-使用PythonREPLTool工具" data-toc-modified-id="7.1.2-使用PythonREPLTool工具-1.2">7.1.2 使用PythonREPLTool工具</a></span>
            </ul>   
    <li><span><a href="#7.2-定义自己的工具并在代理中使用" data-toc-modified-id="7.2-定义自己的工具并在代理中使用-2">7.2 定义自己的工具并在代理中使用</a></span>
        <ul class="toc-item">
            <li><span><a href="#7.2.1-创建和使用自定义时间工具" data-toc-modified-id="7.2.1-创建和使用自定义时间工具-2.1">7.2.1 创建和使用自定义时间工具</a></span></li>
        </ul>
        </li>
    </ul>
</div>


大语言模型学习并记住许多的网络公开信息，大语言模型最常见的应用场景是，将它当作知识库，让它对给定的问题做出回答。

另一种思路是将大语言模型当作推理引擎，让它基于已有的知识库，并利用新的信息（新的大段文本或者其他信息）来帮助回答问题或者进行推理LongChain的内置代理工具便是适用该场景。

本节我们将会了解什么是代理，如何创建代理， 如何使用代理，以及如何与不同类型的工具集成，例如搜索引擎。

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

## 7.1 LangChain内置工具

In [2]:
# 如果需要查看安装过程日志，可删除 -q 
# -U 安装到最新版本的 wikipedia. 其功能同 --upgrade 
!pip install -U -q wikipedia
!pip install -q openai

In [3]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

### 7.1.1 使用llm-math和wikipedia工具

#### 1️⃣ 初始化大语言模型

- 默认密钥`openai_api_key`为环境变量`OPENAI_API_KEY`。因此在运行以下代码之前，确保你已经设置环境变量`OPENAI_API_KEY`。如果还没有密钥，请[获取你的API Key](https://platform.openai.com/account/api-keys) 。
- 默认模型`model_name`为`gpt-3.5-turbo`。
- 更多关于模型默认参数请查看[这里](https://github.com/hwchase17/langchain/blob/master/langchain/chat_models/openai.py)。

In [4]:
# 参数temperature设置为0.0，从而减少生成答案的随机性。
llm = ChatOpenAI(temperature=0)

#### 2️⃣ 加载工具包
- `llm-math` 工具结合语言模型和计算器用以进行数学计算
- `wikipedia`工具通过API连接到wikipedia进行搜索查询。

In [5]:
tools = load_tools(
    ["llm-math","wikipedia"], 
    llm=llm #第一步初始化的模型
)

#### 3️⃣ 初始化代理

- `agent`: 代理类型。这里使用的是`AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION`。其中`CHAT`代表代理模型为针对对话优化的模型，`REACT`代表针对REACT设计的提示模版。
- `handle_parsing_errors`: 是否处理解析错误。当发生解析错误时，将错误信息返回给大模型，让其进行纠正。
- `verbose`: 是否输出中间步骤结果。

In [6]:
agent= initialize_agent(
    tools, #第二步加载的工具
    llm, #第一步初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)

#### 4️⃣.1️⃣ 使用代理回答数学问题

In [7]:
agent("计算300的25%，思考过程请使用中文。") 



> Entering new  chain...


Question: 计算300的25%
Thought: 我可以使用计算器来计算这个百分比
Action:
```
{
  "action": "Calculator",
  "action_input": "300 * 25 / 100"
}
```

Observation: Answer: 75.0
Thought:我现在知道最终答案了
Final Answer: 300的25%是75.0

> Finished chain.


{'input': '计算300的25%，请用中文', 'output': '300的25%是75.0'}

✅ **总结**

1. 模型对于接下来需要做什么，给出思考（Thought） 
    
   <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>：我们需要计算300的25%，这个过程中需要用到乘法和除法。</p>

2. 模型基于思考采取行动（Action）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>行动</strong>: 使用计算器（calculator），输入300*0.25</p>
3. 模型得到观察（Observation）
    <p style="font-family:verdana; font-size:12px;color:green"><strong>观察</strong>：答案: 75.0</p>

4. 基于观察，模型对于接下来需要做什么，给出思考（Thought）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>: 我们的问题有了答案 </p>

5. 给出最终答案（Final Answer）
     <p style="font-family:verdana; font-size:12px;color:green"> <strong>最终答案</strong>: 75.0 </p>
5. 以字典的形式给出最终答案。

#### 4️⃣.2️⃣ Tom M. Mitchell的书

In [18]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
agent(question) 



> Entering new  chain...
I need to find out what book Tom M. Mitchell wrote.
Action: Search for Tom M. Mitchell's books.
Action Input: "Tom M. Mitchell books"
Observation: Search for Tom M. Mitchell's books. is not a valid tool, try another one.
Thought:I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

...

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

> Finished chain.


{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

✅ **总结**

1. 模型对于接下来需要做什么，给出思考（Thought） 
   <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>：我应该使用维基百科去搜索。</p>

2. 模型基于思考采取行动（Action）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>行动</strong>: 使用维基百科，输入Tom M. Mitchell</p>
3. 模型得到观察（Observation）
    <p style="font-family:verdana; font-size:12px;color:green"><strong>观测</strong>: 页面: Tom M. Mitchell，页面: Tom Mitchell (澳大利亚足球运动员)</p>

4. 基于观察，模型对于接下来需要做什么，给出思考（Thought）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>: Tom M. Mitchell写的书是Machine Learning </p>

5. 给出最终答案（Final Answer）
     <p style="font-family:verdana; font-size:12px;color:green"> <strong>最终答案</strong>: Machine Learning </p>
5. 以字典的形式给出最终答案。


值得注意的是，模型每次运行推理的过程可能存在差异，但最终的结果一致。

### 7.1.2 使用PythonREPLTool工具

#### 1️⃣ 创建pyhon代理

In [8]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool

agent = create_python_agent(
    llm,  #使用前面一节已经加载的大语言模型
    tool=PythonREPLTool(), #使用Python交互式环境工具（REPLTool）
    verbose=True #输出中间步骤
)

#### 2️⃣ 使用代理对顾客名字进行排序

In [14]:
customer_list = ["小明","小黄","小红","小蓝","小橘","小绿",]

In [22]:
!pip install -q pinyin

In [23]:
agent.run(f"""在这些客户名字转换为拼音\
并打印输出列表： {customer_list}\
思考过程请使用中文。""") 



> Entering new  chain...


我需要使用拼音库来将客户名字转换为拼音。我可以使用Python的pinyin库来实现这个功能。
Action: Python_REPL
Action Input: import pinyin
Observation: 
Thought:我现在可以使用pinyin库来将客户名字转换为拼音了。
Action: Python_REPL
Action Input: names = ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']
Observation: 
Thought:我现在可以使用pinyin库将names列表中的客户名字转换为拼音了。
Action: Python_REPL
Action Input: pinyin_names = [pinyin.get(i, format='strip', delimiter='') for i in names]
Observation: 
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

我已经成功将客户名字转换为拼音并存储在pinyin_names列表中了。
Final Answer: pinyin_names = ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaojv', 'xiaolv']

> Finished chain.


"pinyin_names = ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaojv', 'xiaolv']"

#### 3️⃣ 使用调试模式

在调试（debug）模式下再次运行，我们可以把上面的6步分别对应到下面的具体流程
1. 模型对于接下来需要做什么，给出思考（Thought）
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/start] [1:chain:AgentExecutor] Entering Chain run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] [12.25s] Exiting LLM run with output</p>
    - <p style="font-family:verdana; font-size:12px;color:green">[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] [12.25s] Exiting Chain run with output</p>
2. 模型基于思考采取行动（Action), 因为使用的工具不同，Action的输出也和之前有所不同，这里输出的为python代码
    - <p style="font-family:verdana; font-size:12px;color:green"> [tool/start] [1:chain:AgentExecutor > 4:tool:Python REPL] Entering Tool run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [tool/end] [1:chain:AgentExecutor > 4:tool:Python REPL] [2.2239999999999998ms] Exiting Tool run with output</p>
3. 模型得到观察（Observation）   
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/start] [1:chain:AgentExecutor > 5:chain:LLMChain] Entering Chain run with input</p>
4. 基于观察，模型对于接下来需要做什么，给出思考（Thought）   
    - <p style="font-family:verdana; font-size:12px;color:green"> [llm/start] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] Entering LLM run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [llm/end] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] [6.94s] Exiting LLM run with output</p>
    
5. 给出最终答案（Final Answer） 
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/end] [1:chain:AgentExecutor > 5:chain:LLMChain] [6.94s] Exiting Chain run with output</p>
6. 返回最终答案。
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/end] [1:chain:AgentExecutor] [19.20s] Exiting Chain run with output</p>

In [25]:
import langchain
langchain.debug=True
agent.run(f"""在这些客户名字转换为拼音\
并打印输出列表： {customer_list}\
思考过程请使用中文。""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "在这些客户名字转换为拼音并打印输出列表： ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']思考过程请使用中文。"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "在这些客户名字转换为拼音并打印输出列表： ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']思考过程请使用中文。",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just retu

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

[llm/end] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] [12.76s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "我现在可以使用pinyin库来将客户名字转换为拼音。\nAction: Python_REPL\nAction Input: names = ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']\npinyin_names = [pinyin.get(i, format='strip', delimiter='') for i in names]\nprint(pinyin_names)",
        "generation_info": null,
        "message": {
          "content": "我现在可以使用pinyin库来将客户名字转换为拼音。\nAction: Python_REPL\nAction Input: names = ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']\npinyin_names = [pinyin.get(i, format='strip', delimiter='') for i in names]\nprint(pinyin_names)",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 379,
      "completion_tokens": 89,
      "total_tokens": 468
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 5:chain:LLMChain] [12.77s] Exiti

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

[llm/end] [1:chain:AgentExecutor > 8:chain:LLMChain > 9:llm:ChatOpenAI] [20.25s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "我现在知道了最终答案。\nFinal Answer: ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaoju', 'xiaolv']",
        "generation_info": null,
        "message": {
          "content": "我现在知道了最终答案。\nFinal Answer: ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaoju', 'xiaolv']",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 500,
      "completion_tokens": 43,
      "total_tokens": 543
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 8:chain:LLMChain] [20.25s] Exiting Chain run with output:
{
  "text": "我现在知道了最终答案。\nFinal Answer: ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaoju', 'xiaolv']"
}
[chain/end] [1:chain:AgentExecutor] [35.93s] Exiting Chain run with output:
{
  "o

## 7.2 定义自己的工具并在代理中使用

In [26]:
# 如果你需要查看安装过程日志，可删除 -q 
!pip install -q DateTime

### 7.2.1 创建和使用自定义时间工具

In [27]:
# 导入tool函数装饰器
from langchain.agents import tool
from datetime import date

#### 1️⃣ 使用tool函数装饰器构建自定义工具
tool函数装饰器可以应用用于任何函数，将函数转化为LongChain工具，使其成为代理可调用的工具。

我们需要给函数加上非常详细的文档字符串, 使得代理知道在什么情况下、如何使用该函数/工具。

比如下面的函数`time`,我们加上了详细的文档字符串

```python
"""
返回今天的日期，用于任何与获取今天日期相关的问题。
输入应该始终是一个空字符串，该函数将始终返回今天的日期。
任何日期的计算应该在此函数之外进行。
"""

```

In [28]:
@tool
def time(text: str) -> str:
    """
    返回今天的日期，用于任何需要知道今天日期的问题。\
    输入应该总是一个空字符串，\
    这个函数将总是返回今天的日期，任何日期计算应该在这个函数之外进行。
    """
    return str(date.today())

#### 2️⃣ 初始化代理

In [36]:
agent= initialize_agent(
    tools=[time], #将刚刚创建的时间工具加入代理
    llm=llm, #初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)

#### 3️⃣ 使用代理询问今天的日期
**注**: 代理有时候可能会出错（该功能正在开发中）。如果出现错误，请尝试再次运行它。

In [37]:
agent("今天的日期是？") 



> Entering new  chain...
根据提供的工具，我们可以使用`time`函数来获取今天的日期。

Thought: 使用`time`函数来获取今天的日期。

Action:
```
{
  "action": "time",
  "action_input": ""
}
```


Observation: 2023-07-04
Thought:我现在知道了最终答案。
Final Answer: 今天的日期是2023-07-04。

> Finished chain.


{'input': '今天的日期是？', 'output': '今天的日期是2023-07-04。'}

✅ **总结**

1. 模型对于接下来需要做什么，给出思考（Thought） 
   <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>：我需要使用 time 工具来获取今天的日期</p>

2. 模型基于思考采取行动（Action), 因为使用的工具不同，Action的输出也和之前有所不同，这里输出的为python代码
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>行动</strong>: 使用time工具，输入为空字符串</p>
    
   
3. 模型得到观察（Observation）
    <p style="font-family:verdana; font-size:12px;color:green"><strong>观测</strong>: 2023-07-04</p>
    
   
    
4. 基于观察，模型对于接下来需要做什么，给出思考（Thought）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>: 我已成功使用 time 工具检索到了今天的日期</p>

5. 给出最终答案（Final Answer）
     <p style="font-family:verdana; font-size:12px;color:green"> <strong>最终答案</strong>: 今天的日期是2023-07-04.</p>
6. 返回最终答案。